In [3]:
from langchain.document_loaders import PyPDFLoader
import os 
import chromadb 
from langchain.vectorstores import Chroma 
from langchain.document_transformers import LongContextReorder 
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceBgeEmbeddings 
from langchain.retrievers.merger_retriever import MergerRetriever 
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter

In [4]:
loader_un_sdg = PyPDFLoader("data/UN SDG.pdf")
documents_un_sdg = loader_un_sdg.load()
text_splitter_un_sdg = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
text_un_sdg = text_splitter_un_sdg.split_documents(documents_un_sdg)

In [5]:

model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

c:\Users\Administrator\miniconda3\envs\idk\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:

retriever = FAISS.from_documents(text_un_sdg, hf).as_retriever()

In [ ]:

docs = retriever.invoke("Explain about the contents in the document")

In [ ]:

# Helper function for printing docs

def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [ ]:
pretty_print_docs(docs)

In [ ]:
model_name = "HuggingFaceH4/zephyr-7b-beta"

import torch
from transformers import ( AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, )
from langchain import HuggingFacePipeline

In [ ]:
#function for loading 4-bit quanrized models 

def load_quantized_model(model_name: str):
    """
    model_name: Name or path of the model to be loaded.
    return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config,
    )
    return model


In [ ]:
# initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    model_name: Name or path of the model for tokenizer initialization.
    return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name, return_token_type_ids=False)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

In [ ]:
tokenizer = initialize_tokenizer(model_name)

In [ ]:
model = load_quantized_model(model_name)

In [ ]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=2048,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

In [ ]:
llm = HuggingFacePipeline(pipeline=pipeline)


In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
normal_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever= retriever
)
 

In [ ]:
esponse1 = normal_chain.invoke("What is Abstractive Question Answering?")

In [ ]:

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [ ]:

compressor = LLMChainExtractor.from_llm(llm)

In [ ]:
compression_retriever=ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)
     

In [ ]:
compressed_docs = compression_retriever.invoke("What is Abstractive Compression")
     

In [ ]:
compressed_docs

In [ ]:
compressed_contexts_len = len("\n\n".join([d.page_content for i, d in enumerate(compressed_docs)]))
     

In [ ]:
from langchain.retrievers.document_compressors import  EmbeddingsFilter

In [ ]:
embeddings_filter = EmbeddingsFilter(embeddings=hf)

In [ ]:
compression_retriever3 = ContextualCompressionRetriever(base_compressor=embeddings_filter, base_retriever=retriever)


In [ ]:
compressed_docs4 = compression_retriever3.invoke("What were the top three priorities outlined in the most recent State of the Union address?")
     

In [ ]:
from langchain.chains import RetrievalQA
     

chain = RetrievalQA.from_chain_type(llm=llm, retriever=compression_retriever)
     

In [ ]:
query="What were the top three priorities outlined in the most recent State of the Union address?"
     

chain.invoke(query)